In [121]:
import pandas as pd
import numpy as np

In [122]:
class NodeSplitter:
    
    def split(self, node):
        features = node.features
        target = node.target
        
        for feature_name in features.columns:
            
            feature_target_df = pd.DataFrame()
            feature_target_df[feature_name] = features[feature_name].values
            
            
            feature = features[feature_name]
            threshold = feature.meadian()
            
            filt = feature > threshold
            subset1 = feature.loc[filt]
            
            filt = feature <= threshold
            subset2 = feature.loc[filt]
        

In [123]:
class DecisionTree:
    class SplitInfo:
    
        def __init__(self, feature_name, threshold):
            self.feature_name = feature_name
            self.threshold = threshold
    
    class Node:
        
        def __init__(self, df, target, parent):
            self.df = df
            self.target = target
            self.parent = parent
            self.split_info = None
            
    class Tree:
        
        def __init__(self):
            self.root = None
    
    class Classifier(Tree):
        
        def fit(self, features, target):
            
            root = DecisionTree.Node(features, target, None)
            self.root = root

## Decision tree test

## Loading dataset

In [124]:
report_df = pd.read_csv('../datasets/other/world_happiness_report.csv')
report_df.head()

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440


In [125]:
renamed_columns = {
    'Country Name' : 'country',
    'Regional Indicator' : 'region',
    'Year' : 'year',
    'Life Ladder' : 'life_ladder',
    'Log GDP Per Capita' : 'gdp_per_capita',
    'Social Support' : 'social_support',
    'Healthy Life Expectancy At Birth' : 'healthy_life',
    'Freedom To Make Life Choices' : 'free_to_choose',
    'Generosity' : 'generosity',
    'Perceptions Of Corruption' : 'corruption',
    'Positive Affect' : 'pos_affect',
    'Negative Affect' : 'neg_affect',
    'Confidence In National Government' : 'gov_confidence'
}

report_df.rename(columns=renamed_columns, inplace=True)
report_df.head()

,country,region,year,life_ladder,gdp_per_capita,social_support,healthy_life,free_to_choose,generosity,corruption,pos_affect,neg_affect,gov_confidence
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440


In [126]:
report_df.drop(columns=['country', 'generosity', 'corruption', 'year', \
    'pos_affect', 'neg_affect', 'gov_confidence', 'healthy_life', 'gdp_per_capita'], inplace=True)
report_df.head()

,region,life_ladder,social_support,free_to_choose
0,South Asia,3.723590,0.450662,0.718114
1,South Asia,4.401778,0.552308,0.678896
2,South Asia,4.758381,0.539075,0.600127
3,South Asia,3.831719,0.521104,0.495901
4,South Asia,3.782938,0.520637,0.530935


In [127]:
print(f'Dataset size BEFORE NaN row elimination is {str(len(report_df))}')

feature_median = report_df.social_support.median()
filt = report_df.social_support.isnull()
report_df.loc[filt, 'social_support'] =  feature_median

feature_median = report_df.free_to_choose.median()
filt = report_df.free_to_choose.isnull()
report_df.loc[filt, 'free_to_choose'] =  feature_median

report_df.dropna(inplace=True)

print(f'Dataset size AFTER NaN row elimination is {str(len(report_df))}')

Dataset size BEFORE NaN row elimination is 2199
Dataset size AFTER NaN row elimination is 2087


In [128]:
if (report_df.isnull().mean() == 0).all():
    print('Dataset NaN were values eliminated')

Dataset NaN were values eliminated


In [129]:
classifier = DecisionTree.Classifier()


In [130]:
#Split code
def gini_impurity (value_counts):
    n = value_counts.sum()
    p_sum = 0
    for key in value_counts.keys():
        p_sum = p_sum  +  (value_counts[key] / n ) * (value_counts[key] / n ) 
    gini = 1 - p_sum
    return gini


feature = report_df['social_support']
feature_name = feature.name

target = report_df['region']

feature_target_df = pd.DataFrame()
feature_target_df[feature_name] = feature
feature_target_df[target.name] = target

threshold = feature.median()
columns = [feature_name, target.name]    
    
filt = feature_target_df[feature_name] > threshold
subset1 = pd.DataFrame()
subset1[columns] = feature_target_df.loc[filt][columns].values
            
filt = feature_target_df[feature_name] <= threshold
subset2 = pd.DataFrame()
subset2[columns] = feature_target_df.loc[filt][columns].values

print(f'Dataset lenght is {str(len(feature_target_df))}')
print(f'Subset 1 lenght is {str(len(subset1))}')
print(f'Subset 2 lenght is {str(len(subset2))}')

subset1.head()

Dataset lenght is 2087
Subset 1 lenght is 1043
Subset 2 lenght is 1044


,social_support,region
0,0.839397,Middle East and North Africa
1,0.867649,Middle East and North Africa
2,0.84071,Middle East and North Africa
3,0.938463,Latin America and Caribbean
4,0.862206,Latin America and Caribbean


In [131]:
subset1_gini_index = gini_impurity(subset1[target.name].value_counts())
subset2_gini_index = gini_impurity(subset2[target.name].value_counts())

gini_index = (subset1_gini_index + subset2_gini_index) / 2
gini_index

0.8213555506848795

In [132]:
target.name

'region'